# Numerical comparison of temperature models

Here I will describe and compare two models. The first model uses the IEEE 738 standard, while the second model is the one derived here.

## Summary of IEEE 738 temperature calculation

IEEE recommends numerically integrating the following heat balance equation to compute temperature changes:

\begin{align}
\frac{dT_\text{avg}}{dt} &= \frac{1}{mC_p}\left(I^2R(T_\text{avg}) - q_c - q_r + q_s\right)
\end{align}

The temporal instanton framework uses the DC loss expression rather than $I^2R(T_\text{avg})$. Substitute:

\begin{align}
\frac{dT_\text{avg}}{dt} &= \frac{1}{mC_p}\left( r_{ij}\frac{\theta_{ij}^2}{x_{ij}^2}\frac{S_b}{3L_{ij}} - q_c - q_r + q_s\right)
\end{align}

Heat rates $q_c$ and $q_r$ are calculated as follows:

\begin{align}
q_c &= \eta_c\cdot(T - T_\text{amb}) \\
q_r &= \eta_r\cdot((T + 273)^4 - (T_\text{amb} + 273)^4)
\end{align}

All other parameters are constant. The important thing to keep in mind about the IEEE 738 model is that it requires the user to select an integration time step. IEEE recommends a step size less than 10% of the conductor thermal time constant (which is on the order of ten minutes). A smaller integration step size will yield more accurate results.

## Summary of approximate temperature calculation

As discussed previously, we can linearize the radiative heat loss rate to obtain an analytic expression for temperature:

\begin{align}
T(t_n) &= f + \frac{c}{a}\left[ \sum_{i=1}^n \left( (e^{(t_1-t_0)a})^i - (e^{(t_1-t_0)a})^{i-1} \right)\theta_{ij}(t_{n-i+1})^2\right]
\end{align}

where

* $\hat{\theta}_{ij}(t_{k})=  \theta_{ij}(t_k)\sqrt{ (e^{(t_1-t_0)a})^{n-k+1} - (e^{(t_1-t_0)a})^{n-k} } $
    * $\theta_{ij}(t_k)$ is the angle difference across line $ij$ at time interval $t_k$.
    * $t_1-t_0$ is the length of each time interval (in seconds)
* $a = \frac{1}{mC_p}\left[ -\eta_c - 4\eta_r(T_\text{mid} + 273)^3 \right]$ has units of $s^{-1}$
    * $mC_p$ is the heat capacity, with units of J/m-$^\circ$C
    * $\eta_c$ is the conductive heat loss rate coefficient, with units of W/m-$^\circ$C
    * $\eta_r$ is the conductive heat loss rate coefficient, with units of W/m-$^\circ$C$^4$
    * $T_\text{mid}$ is the average of ambient temperature $T_\text{amb}$ and limit temperature $T_\text{lim}$, in Celsius
* $c = \frac{r_{ij}S_b}{3 mC_p x_{ij}^2L_{ij}}$ is a constant with units of W/m
    * $r_{ij}$ is resistance of line $ij$ in per unit
    * $x_{ij}$ is reactance of line $ij$ in per unit
    * $S_b$ is the system base (e.g. 100 MVA)
    * $L_{ij}$ is the length of one phase of line $ij$, in m
* $f = (e^{(t_1 - t_0)a})^n T(t_0) + \frac{d}{a}\left[ \sum_{i=1}^n \left( (e^{(t_1-t_0)a})^i - (e^{(t_1-t_0)a})^{i-1} \right)\right]$ is a constant with units of degrees Celsius
    * $T(t_0)$ is the line's initial temperature (steady state temperature under base case power flow condition)
    * $d = \frac{1}{mC_p}\left[ \eta_cT_\text{amb} - \eta_r\left((T_\text{mid} + 273)^4 - (T_\text{amb} + 273)^4\right) + 4\eta_rT_\text{mid}(T_\text{mid} + 273)^3 + q_s \right]$ 
    
    is a constant with units of W/m
        * $q_s$ is the solar heat gain rate in W/m

## Comparison

I used RTS-96 network data and Waxwing conductor parameters to compare the two line temperature models. The figure below shows line temperatures calculated across three ten-minute time intervals, where each interval has a different angle difference (power flow). The angle differences are

* Interval 1: 0.09 rad
* Interval 2: 0.04 rad
* Interval 3: 0.15 rad

The blue trace comes from the IEEE 738 temperature model with a 1s integration step size. The green trace comes from my approximate model (with linearized radiative heat loss). The green dot is the final temperature computed directly with the approximate model temperature expression.

<img src="../images/temp_model_comparison2.png" width=600px>

Notes:

* Because the approximate line temperature model is analytic (IVP solution) while the 738 model is numeric (Euler integration), I chose a very small integration step size (one second) for the 738 temperature calculation to facilitate comparison.

* Because the approximate model underestimates the radiative heat loss rate, the green trace should lie slightly above the blue one. This makes the approximate model conservative, which is desirable in the temporal instanton setting. As the figure shows, the final temperature computed by the approximate model is indeed slightly higher than that computed by the IEEE 738 method.